# 42. CNN - Alexnet

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

### 텐서플로 실습시간 그 다섯번째!<br>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. Alexnet을 구현해본다. 

#### 사용데이터.
1. Multinomial Classification : http://yann.lecun.com/exdb/mnist/


![image](http://mblogthumb1.phinf.naver.net/20160314_204/laonple_14579300603930fQ7q_PNG/%C0%CC%B9%CC%C1%F6_2.png?type=w2)


In [ ]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

In [ ]:
def weight(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.01)
    w = tf.Variable(initial, name=name)
    tf.add_to_collection('weights', w)
    return w

def bias(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial, name=name)

def conv2d(x, W, stride, padding):
    return tf.nn.conv2d(x, W, strides=[1, stride[0], stride[1], 1], padding=padding)

def max_pool2d(x, kernel, stride, padding):
    return tf.nn.max_pool(x, ksize=kernel, strides=stride, padding=padding)

def lrn(x, depth_radius, bias, alpha, beta):
    return tf.nn.local_response_normalization(x, depth_radius, bias, alpha, beta)

def relu(x):
    return tf.nn.relu(x)

def batch_norm(x):
    epsilon = 0.001
    batch_mean, batch_var = tf.nn.moments(x, [0])
    return tf.nn.batch_normalization(x, batch_mean, batch_var, None, None, epsilon)

In [ ]:
# Graph Clear
tf.reset_default_graph()

In [ ]:
lr = tf.placeholder(tf.float32)
kp = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])


In [ ]:
with tf.name_scope('conv1') as inner_scope:
    wcnn1 = weight([5, 5, 1, 48], name='wcnn1')
    bcnn1 = bias([48], name='bcnn1')
    conv1 = tf.add(conv2d(X, wcnn1, stride=(4, 4), padding='SAME'), bcnn1)
    conv1 = batch_norm(conv1)
    conv1 = relu(conv1)
    norm1 = lrn(conv1, depth_radius=5, bias=1.0, alpha=1e-04, beta=0.75)
    pool1 = max_pool2d(norm1, kernel=[1, 3, 3, 1], stride=[1, 2, 2, 1], padding='SAME')
    
with tf.name_scope('conv2') as inner_scope:
    wcnn2 = weight([3, 3, 48, 128], name='wcnn2')
    bcnn2 = bias([128], name='bcnn2')
    conv2 = tf.add(conv2d(pool1, wcnn2, stride=(1, 1), padding='SAME'), bcnn2)
    conv2 = batch_norm(conv2)
    conv2 = relu(conv2)
    norm2 = lrn(conv2, depth_radius=5, bias=1.0, alpha=1e-04, beta=0.75)
    pool2 = max_pool2d(norm2, kernel=[1, 3, 3, 1], stride=[1, 2, 2, 1], padding='SAME')

with tf.name_scope('conv3') as inner_scope:
    wcnn3 = weight([3, 3, 128, 192], name='wcnn3')
    bcnn3 = bias([192], name='bcnn3')
    conv3 = tf.add(conv2d(pool2, wcnn3, stride=(1, 1), padding='SAME'), bcnn3)
    conv3 = batch_norm(conv3)
    conv3 = relu(conv3)

with tf.name_scope('conv4') as inner_scope:
    wcnn4 = weight([3, 3, 192, 192], name='wcnn4')
    bcnn4 = bias([192], name='bcnn4')
    conv4 = tf.add(conv2d(conv3, wcnn4, stride=(1, 1), padding='SAME'), bcnn4)
    conv4 = batch_norm(conv4)
    conv4 = relu(conv4)

with tf.name_scope('conv5') as inner_scope:
    wcnn5 = weight([3, 3, 192, 128], name='wcnn5')
    bcnn5 = bias([128], name='bcnn5')
    conv5 = tf.add(conv2d(conv4, wcnn5, stride=(1, 1), padding='SAME'), bcnn5)
    conv5 = batch_norm(conv5)
    conv5 = relu(conv5)
    pool5 = max_pool2d(conv5, kernel=[1, 3, 3, 1], stride=[1, 2, 2, 1], padding='SAME')
    
dim = pool5.get_shape().as_list()
flat_dim = dim[1] * dim[2] * dim[3] # 6 * 6 * 256
flat = tf.reshape(pool5, [-1, flat_dim])

with tf.name_scope('fullyconected1') as inner_scope:
    wfc1 = weight([flat_dim, 4096], name='wfc1')
    bfc1 = bias([4096], name='bfc1')
    fc1 = tf.add(tf.matmul(flat, wfc1), bfc1)
    fc1 = batch_norm(fc1)
    fc1 = relu(fc1)
    fc1 = tf.nn.dropout(fc1, kp)

with tf.name_scope('fullyconected2') as inner_scope:
    wfc2 = weight([4096, 1000], name='wfc2')
    bfc2 = bias([1000], name='bfc2')
    fc2 = tf.add(tf.matmul(fc1, wfc2), bfc2)
    fc2 = batch_norm(fc2)
    fc2 = relu(fc2)
    fc2 = tf.nn.dropout(fc2, kp)

with tf.name_scope('classifier_output') as inner_scope:
    wfc3 = weight([1000, 10], name='wfc3')
    bfc3 = bias([10], name='bfc3')
    pred = tf.add(tf.matmul(fc2, wfc3), bfc3)
    softmax = tf.nn.softmax(pred)


In [ ]:
LAMBDA = 0.00005 # for weight decay

# cross-entropy and weight decay
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y, name='cross-entropy'))

l2_loss = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in tf.get_collection('weights')]))
tf.summary.scalar('l2_loss', l2_loss)

loss = cross_entropy + l2_loss
tf.summary.scalar('loss', loss)

# accuracy
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# momentum optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


In [ ]:
# 세션 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
import time 

learning_rate = 0.01
epochs = 1
batch_size = 200
dropout = 0.7

num_batches = int(mnist.train.num_examples / batch_size)
display_step = 2 * int(2500 / batch_size)

start_time = time.time()

for e in range(epochs):
    print("%dth epoch - total batches %d" % ((e + 1), num_batches))
    
    for i in range(num_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X: X_batch.reshape(-1, 28, 28, 1), 
                                       Y: Y_batch, 
                                       lr: learning_rate, 
                                       kp: dropout})

        # display current training informations
        if (i + 1) % display_step == 0:
            temp_time=time.time()
            c, a = sess.run([loss, accuracy], feed_dict={X: X_batch.reshape(-1, 28, 28, 1), 
                                                         Y: Y_batch, 
                                                         lr: learning_rate, 
                                                         kp: 1.0})
            
            log = "time: {:.2f} Step/Batch: {} --- Loss: {:.4f} Training accuracy: {:.4f}"
            print(log.format((temp_time - start_time), (i + 1), c, a))

    v_a = sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), 
                                        Y: mnist.test.labels, 
                                        lr: learning_rate, 
                                        kp: 1.0})
    # intermediate time
    int_time = time.time()
    print('Elapsed time: {:.04f}'.format(int_time - start_time))
    print('Validation accuracy: {:.04f}'.format(v_a))

end_time = time.time()
print('Elapsed time: {:.04f}'.format(end_time - start_time))


### 실습 1
위의 alexnet을 모델을 보고 제대로 구성하여. 성능 튜닝을 해봅시다. 

## CIFAR-10 + alexnet

In [ ]:
import tensorflow as tf
from tbij import show_graph # Tensor Board In Jupyter notebook. You can see this later.

import cifar10
cifar10.data_path = "./CIFAR-10/"
# cifar10.maybe_download_and_extract()
class_names = cifar10.load_class_names()

images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

############################
# Place Holders
lr = tf.placeholder(tf.float32)
kp = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
Y = tf.placeholder(tf.float32, [None, 10], name="Y")

### 실습 2
위의 alexnet을 모델로 cifar-10을 분류하는 모델을 만들어보자.

In [ ]:
'''
실습 2
alexnet 구성하기
'''
tf.reset_default_graph()

lr = tf.placeholder(tf.float32)
kp = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])

with tf.name_scope('conv1') as inner_scope:
    wcnn1 = weight([5, 5, 3, 48], name='wcnn1')
    bcnn1 = bias([48], name='bcnn1')
    conv1 = tf.add(conv2d(X, wcnn1, stride=(4, 4), padding='SAME'), bcnn1)
    conv1 = batch_norm(conv1)
    conv1 = relu(conv1)
    norm1 = lrn(conv1, depth_radius=5, bias=1.0, alpha=1e-04, beta=0.75)
    pool1 = max_pool2d(norm1, kernel=[1, 3, 3, 1], stride=[1, 2, 2, 1], padding='SAME')
    
with tf.name_scope('conv2') as inner_scope:
    wcnn2 = weight([3, 3, 48, 128], name='wcnn2')
    bcnn2 = bias([128], name='bcnn2')
    conv2 = tf.add(conv2d(pool1, wcnn2, stride=(1, 1), padding='SAME'), bcnn2)
    conv2 = batch_norm(conv2)
    conv2 = relu(conv2)
    norm2 = lrn(conv2, depth_radius=5, bias=1.0, alpha=1e-04, beta=0.75)
    pool2 = max_pool2d(norm2, kernel=[1, 3, 3, 1], stride=[1, 2, 2, 1], padding='SAME')

with tf.name_scope('conv3') as inner_scope:
    wcnn3 = weight([3, 3, 128, 192], name='wcnn3')
    bcnn3 = bias([192], name='bcnn3')
    conv3 = tf.add(conv2d(pool2, wcnn3, stride=(1, 1), padding='SAME'), bcnn3)
    conv3 = batch_norm(conv3)
    conv3 = relu(conv3)

with tf.name_scope('conv4') as inner_scope:
    wcnn4 = weight([3, 3, 192, 192], name='wcnn4')
    bcnn4 = bias([192], name='bcnn4')
    conv4 = tf.add(conv2d(conv3, wcnn4, stride=(1, 1), padding='SAME'), bcnn4)
    conv4 = batch_norm(conv4)
    conv4 = relu(conv4)

with tf.name_scope('conv5') as inner_scope:
    wcnn5 = weight([3, 3, 192, 128], name='wcnn5')
    bcnn5 = bias([128], name='bcnn5')
    conv5 = tf.add(conv2d(conv4, wcnn5, stride=(1, 1), padding='SAME'), bcnn5)
    conv5 = batch_norm(conv5)
    conv5 = relu(conv5)
    pool5 = max_pool2d(conv5, kernel=[1, 3, 3, 1], stride=[1, 2, 2, 1], padding='SAME')
    
dim = pool5.get_shape().as_list()
flat_dim = dim[1] * dim[2] * dim[3] # 6 * 6 * 256
flat = tf.reshape(pool5, [-1, flat_dim])

with tf.name_scope('fullyconected1') as inner_scope:
    wfc1 = weight([flat_dim, 4096], name='wfc1')
    bfc1 = bias([4096], name='bfc1')
    fc1 = tf.add(tf.matmul(flat, wfc1), bfc1)
    fc1 = batch_norm(fc1)
    fc1 = relu(fc1)
    fc1 = tf.nn.dropout(fc1, kp)

with tf.name_scope('fullyconected2') as inner_scope:
    wfc2 = weight([4096, 1000], name='wfc2')
    bfc2 = bias([1000], name='bfc2')
    fc2 = tf.add(tf.matmul(fc1, wfc2), bfc2)
    fc2 = batch_norm(fc2)
    fc2 = relu(fc2)
    fc2 = tf.nn.dropout(fc2, kp)

with tf.name_scope('classifier_output') as inner_scope:
    wfc3 = weight([1000, 10], name='wfc3')
    bfc3 = bias([10], name='bfc3')
    pred = tf.add(tf.matmul(fc2, wfc3), bfc3)
    softmax = tf.nn.softmax(pred)


In [ ]:
LAMBDA = 0.00005 # for weight decay

# cross-entropy and weight decay
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y, name='cross-entropy'))

l2_loss = tf.reduce_sum(LAMBDA * tf.stack([tf.nn.l2_loss(v) for v in tf.get_collection('weights')]))
tf.summary.scalar('l2_loss', l2_loss)

loss = cross_entropy + l2_loss
tf.summary.scalar('loss', loss)

# accuracy
correct = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
tf.summary.scalar('accuracy', accuracy)

# momentum optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)


In [ ]:
# 세션 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())
times = 1

In [ ]:
import time 

learning_rate = 0.005
epochs = 5
batch_size = 200
dropout = 0.7
num_batches = int(len(images_train) / batch_size)
display_step = int(2500 / batch_size)

start_time = time.time()

for e in range(epochs):
    print("%dth epoch - total batches %d" % ((e + times), num_batches))
    
    for i in range(num_batches):
        X_batch = images_train[i * batch_size:(i + 1) * batch_size]
        Y_batch = labels_train[i * batch_size:(i + 1) * batch_size]
        sess.run(optimizer, feed_dict={X: X_batch, 
                                       Y: Y_batch, 
                                       lr: learning_rate, 
                                       kp: dropout})

        # display current training informations
        if (i + 1) % display_step == 0:
            temp_time=time.time()
            c, a = sess.run([loss, accuracy], feed_dict={X: X_batch, 
                                                         Y: Y_batch, 
                                                         lr: learning_rate, 
                                                         kp: 1.0})
            
            log = "time: {:.2f} Step/Batch: {} --- Loss: {:.4f} Training accuracy: {:.4f}"
            print(log.format((temp_time - start_time), (i + 1), c, a))

    v_a = sess.run(accuracy, feed_dict={X: images_test, 
                                        Y: labels_test, 
                                        lr: learning_rate, 
                                        kp: 1.0})
    # intermediate time
    int_time = time.time()
    print('Elapsed time: {:.04f}'.format(int_time - start_time))
    print('Validation accuracy: {:.04f}'.format(v_a))

times += epochs
end_time = time.time()
print('Elapsed time: {:.04f}'.format(end_time - start_time))
